<a href="https://colab.research.google.com/github/IFuentesSR/AET_moderate_eval/blob/main/AET_synth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='upbeat-imprint-269809')

In [ ]:
yyKey = ee.List.sequence(1982, 2019).map(parse)
yyVal = ee.List.sequence(1982, 2019)
yyDic = ee.Dictionary.fromLists(yyKey, yyVal)
mmDic = ee.Dictionary({'01':1,'02':2,'03':3,'04':4,'05':5,'06':6,'07':7,'08':8,'09':9,'10':10,'11':11,'12':12})

In [ ]:
basins = ee.FeatureCollection('xxxxxxxxxxxxxxxxxxxx')

In [ ]:
def parse(i):
    return ee.Number(i).int().format()


def readAsset(image):
    id = image.id().split("_");
    yy = ee.Number(yyDic.get(id.get(1)));
    mm = ee.Number(mmDic.get(id.get(2)));
    date = ee.Date.fromYMD(day=1, month=mm, year=yy).millis()
    return image.divide(100).rename('SynthesizedET').setMulti({'mm':mm, 'yy':yy, 'system:time_start':date})


def AET_SYN(collection):
    def wrap(fea):
        def inner(img):
            ET = img.reduceRegion('mean', fea.geometry(), 1000).values().get(0)
            return ee.Feature(None, {'AET':ET,
                                    'Cuenca':fea.get('NOM_CUEN'),
                                    'date':ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')})

        return collection.map(inner)
    return wrap

In [ ]:
SynthesizedET = ee.ImageCollection('users/Elnashar/SynthesizedET').map(readAsset).filterDate('2003-01-01', '2020-01-01')
AET_Syn = basins.map(AET_SYN(SynthesizedET)).flatten()
ee.batch.Export.table.toDrive(AET_Syn, 'xxxx').start()